In [149]:
# import block
from gensim.models import word2vec
import pandas as pd
import json
from pathlib import Path
import string
import re
import pickle
from itertools import chain
from fuzzywuzzy import fuzz

In [3]:
#first load all recipe data
BOrecipes = []
BOingredients = []
BOids = []
BOpath = Path('~/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/')
for i, po in enumerate(BOpath.expanduser().glob('*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        BOrecipes.append(r)
        BOing = r.get('Ingredients')
        BOid = r.get('RecipeID')
        BOids.append(id)
        if BOing is not None:
            #BOingredients.append([i['Name'].strip(string.punctuation).lower().strip(string.digits).strip() for i in BOing if i is not None and i['Name'] is not None])
            BOingredients.append([re.sub(r'[^a-z ]', '', i['Name'].lower().strip()) for i in BOing if i is not None and i['Name'] is not None])

/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-68-87332.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-71-87778.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-19-156227.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-127-74-2431270.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-112-159-862538.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-62-166860.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-35-191-157959.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-76-83346.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-74-82-565669.json
/Users/bridgitboulahanis/Dropbox

In [141]:
#now we want to load spoonacular data and create a dataframe that has ingredients and their aisle 
# this will be our classification data

SArecipes = []
SAingredients = []
SAingclass = []
SAnoise = []
SAingID = []
SAmeta = []
SAmeasure = []

seenID = set()
SApath = Path('~/Dropbox/Not_work/insight/greenmarkets/spoonac/data/')
for i, po in enumerate(SApath.expanduser().glob('*/*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        for rec in r:
            currenting = rec['extendedIngredients']
            for ing in currenting:
                SAnoise.extend(i.strip() for i in ing['metaInformation'])
                if ing['id'] in seenID:
                    continue
                seenID.add(ing['id']) 
                SAingredients.append(ing['name'])
                SAingclass.append(ing['aisle'])  
                SAingID.append(ing['id'])
                SAmeasure.append(ing['measures']['us']['unitLong'])
                SAmeta.extend([ing['meta']])

                    





In [152]:
#grab all the ingrdients from spoonacular and put in dataframe
classifying = pd.DataFrame({'ingredient': SAingredients, 'aisle': SAingclass, 'meta': SAmeta, 'measure': SAmeasure,  'id': SAingID})
classifying

,ingredient,aisle,meta,measure,id
0,bell pepper,Produce,"[seeded, chopped]",large,10211821.0
1,canned coconut milk,Canned and Jarred,[canned],ounces,12117.0
2,carrots,Produce,"[peeled, chopped]",larges,11124.0
3,cornstarch,Baking,[],Tbsps,20027.0
4,curry powder,Spices and Seasonings,[],Tbsp,2015.0
...,...,...,...,...,...
1949,aji amarillo,Produce;Ethnic Foods,[cut into 1/4-inch dice*],Tbsps,10311819.0
1950,aji amarillo paste,Ethnic Foods,[],teaspoon,1006973.0
1951,yuca,Produce,"[fresh, cut into big pieces]",pound,11134.0
1952,cedar plank,Grilling Supplies;Gourmet,[],,11911111.0


In [153]:
#some words you noticed should get added to SAnoise
wta = ['ground', 'taste', 'ounce']
SAnoise.extend(wta)
SAnoise = set(SAnoise)
#remember you can check if it is in there with: 'fresh' in SAnoise

In [8]:
#suite of deduping functions

#on loading we tokenize, strip punctuation&digits&spaces and make lowercase 

def removenoise(input_ing):
    noise_free_ing = []
    for word in input_ing:
        checked = []
        splitit = word.split()
        checked.extend(i for i in splitit if i not in SAnoise)
        noise_free_ing.append(' '.join(checked))
    return noise_free_ing
        
#stem? lemmatize? part of speech tagging? stop words
    
    

In [154]:
#call the dedupe functions
nning = []
for i in BOingredients:
    nning.append(removenoise(i))



In [58]:
# classify ingredients into aisles
whichaisle = {}
seen = set()
notin = []
for i in nning:
    for j in i:
        if j in seen:
            continue
        seen.add(j) 
        a = classifying.index[classifying['ingredient'] == j].tolist()        
        if len(a) >= 1:
            whichaisle[j] = classifying['aisle'][a].tolist()
        if len(a) == 0:
            whichaisle[j] = None
            notin.append(j)

            # okay so instead of exact matching maybe fuzzy wuzzy


In [156]:
print(len(whichaisle))
#whichaisle
len(notin)

39506


37098

In [33]:
#Actual W2V modeling
# Set values for NN parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 3    # 50% of the corpus                    
num_workers = 4       # Number of CPUs
context = 10          # Context window size; 
downsampling = 1e-3   # threshold for configuring which 
                    # higher-frequency words are randomly downsampled

# Initialize and train the model 
model = word2vec.Word2Vec(nning, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

In [43]:
#output both the w2v model and the aisle classification dict
pickle.dump(model,open("bigoven/w2vmodel.pkl","wb"))
pickle.dump(whichaisle,open("bigoven/aisleclassification.pkl","wb"))
pickle.dump(SAnoise, open('bigoven/noiselist.pkl', 'wb'))

In [41]:
#proof of concept, also not a production cell
print(model.wv.most_similar('feta cheese'))
print('=============')
print(model.wv.most_similar('baby spinach'))

[('kalamata olives', 0.9773502349853516), ('baby spinach', 0.9714789986610413), ('plum tomatoes', 0.9671364426612854), ('pine nuts', 0.9558079838752747), ('sundried tomatoes', 0.952326774597168), ('grape tomatoes', 0.9518359899520874), ('basil leaves', 0.9489792585372925), ('cherry tomatoes', 0.9453437924385071), ('san marzano tomatoes', 0.9453309774398804), ('goat cheese', 0.9413638114929199)]
[('sundried tomatoes', 0.9894789457321167), ('goat cheese', 0.9803232550621033), ('grape tomatoes', 0.9769477844238281), ('kalamata olives', 0.9761931300163269), ('feta cheese', 0.9714789986610413), ('artichoke hearts', 0.9684235453605652), ('pine nuts', 0.9621485471725464), ('orzo pasta', 0.9534425735473633), ('grilled chicken breasts', 0.9517673850059509), ('oil cured olives', 0.9511600732803345)]


In [151]:
fuzz.ratio('chicken', 'olive')

33